In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import sys
sys.path.append(os.path.normpath(os.path.join(os.getcwd(), '..')))
os.environ['THEANO_FLAGS'] = "device=cuda0"

In [ ]:
import shelve
from pprint import pprint
import numpy as np
import lasagne
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib as mpl
import matplotlib.cm
from lproc import subset, rmap
from datasets.utils import gloss2seq, seq2gloss
from sltools.nn_utils import onehot, jaccard

# Reload dataset and model

In [ ]:
# from experiments.ch14_skel.a_data import durations, gloss_seqs, tmpdir, \
#     train_subset, val_subset, test_subset
# from experiments.ch14_skel.b_preprocess import feat_seqs
# feat_seqs = rmap(lambda x: (x,), feat_seqs)

# from experiments.ch14_bgr.a_data import durations, gloss_seqs, tmpdir, \
#     train_subset, val_subset, test_subset
# from experiments.ch14_bgr.b_preprocess import feat_seqs
# feat_seqs = rmap(lambda x: (x,), feat_seqs)

# from experiments.ch14_fusion.a_data import durations, gloss_seqs, tmpdir, \
#     train_subset, val_subset, test_subset
# from experiments.ch14_fusion.b_preprocess import feat_seqs

from experiments.ch14_transfer2.a_data import durations, gloss_seqs, tmpdir, \
    train_subset, val_subset, test_subset
from experiments.ch14_transfer2.b_preprocess import feat_seqs
feat_seqs = rmap(lambda x: (x,), feat_seqs)

feats_seqs_train = subset(feat_seqs, train_subset)
gloss_seqs_train = subset(gloss_seqs, train_subset)
durations_train = subset(durations, train_subset)

feats_seqs_val = subset(feat_seqs, val_subset)
gloss_seqs_val = subset(gloss_seqs, val_subset)
durations_val = subset(durations, val_subset)

feats_seqs_test = subset(feat_seqs, test_subset)
gloss_seqs_test = subset(gloss_seqs, test_subset)
durations_test = subset(durations, test_subset)

report = shelve.open(os.path.join(tmpdir, "hmm_report"))

# Training

In [ ]:
all_batch_losses = []
all_epoch_losses = []
n_epochs = []
best_phase = 0
best_score = 0
for i in sorted(map(int, report.keys())):
    r = report[str(i)]
    all_batch_losses += r['batch_losses']
    all_epoch_losses += r['epoch_losses']
    if r['val_report']['jaccard'] > best_score:
        best_phase = i
        best_score = r['val_report']['jaccard']

phase_report = report[str(best_phase)]
print("best phase is {} with JI {}".format(best_phase, best_score))
recognizer = phase_report['model']
previous_recognizer = report[str(best_phase - 1)]['model'] if best_phase > 0 else None

pprint(phase_report['settings'])

In [ ]:
plt.figure(figsize=(12, 3))
plt.scatter(np.arange(len(all_batch_losses)), all_batch_losses, marker='.', alpha=.1)
plt.gca().set_yscale("log")
plt.figure(figsize=(12, 3))
plt.plot(np.arange(len(all_epoch_losses)), all_epoch_losses)
plt.gca().set_yscale("log")
plt.show()

# Performances

In [ ]:
def plot_perfs(perf_report, chains_lengths):
    plt.figure(figsize=(14, 7))

    ax = plt.subplot2grid((2, 3), (0, 0))
    cmap = matplotlib.cm.viridis
    cmap.set_bad(cmap(0.001))
    plt.imshow(perf_report['confusion'] / perf_report['confusion'].sum(axis=1, keepdims=True), 
               interpolation='none', 
               cmap=cmap, clim=(0.001, 1), norm=LogNorm(vmin=0.001, vmax=1))
    plt.gca().set_xticklabels([])
    plt.gca().set_yticklabels([])
    ax.set_title("HMM Jaccard/Framewise : {:0.3f}/{:0.3f}".format(
        perf_report['jaccard'], perf_report['framewise']))

    ax = plt.subplot2grid((2, 3), (1, 0))
    cmap = matplotlib.cm.viridis
    cmap.set_bad(cmap(0.001))
    plt.imshow(perf_report['posterior_confusion'] / perf_report['posterior_confusion'].sum(axis=1, keepdims=True), 
               interpolation='none', 
               cmap=cmap, clim=(0.001, 1), norm=LogNorm(vmin=0.001, vmax=1))
    plt.gca().set_xticklabels([])
    plt.gca().set_yticklabels([])
    ax.set_title("Posterior Jaccard/Framewise : {:0.3f}/{:0.3f}".format(
        perf_report['posterior_jaccard'], perf_report['posterior_framewise']))

    ax = plt.subplot2grid((2, 3), (0, 1), colspan=2, rowspan=2)
    cmap = matplotlib.cm.viridis
    cmap.set_bad(cmap(0.001))
    plt.imshow(perf_report['statewise_confusion'] / perf_report['statewise_confusion'].sum(axis=1, keepdims=True), 
               interpolation='none', 
               cmap=cmap, clim=(0.001, 1), norm=LogNorm(vmin=0.001, vmax=1))
    plt.gca().set_xticklabels([])
    plt.gca().set_yticklabels([])
    plt.gca().set_xticks(np.cumsum(chains_lengths) - .5)
    plt.gca().set_yticks(np.cumsum(chains_lengths) - .5)
    plt.gca().grid(color='gray', linestyle='dotted')
    ax.set_title("State-wise framewise: {:0.3f}".format(perf_report['statewise_framewise']))
    
    plt.colorbar()
    
    plt.show()

In [ ]:
plot_perfs(phase_report['train_report'], recognizer.chains_lengths)
plot_perfs(phase_report['val_report'], recognizer.chains_lengths)

# Error

In [ ]:
labels = [gloss2seq(g_, d_, 0) for g_, d_ in zip(gloss_seqs_val, durations_val)]
preds = recognizer.predict(feats_seqs_val)

In [ ]:
# distribution of errors

score = [jaccard(onehot(l, np.arange(1, 20)), onehot(p, np.arange(1, 20)))
                 for l, p in zip (labels, preds)]

plt.hist(score, np.linspace(0.5, 1, 40))
plt.show()

In [ ]:
# nb of false positives out of sequence vocabulary

np.mean([len(set(p_) - set(l_)) for p_, l_ in zip(preds, labels)], axis=0)

In [ ]:
# confusion types

preds_cat = np.concatenate(preds)
labels_cat = np.concatenate(labels)

confusion = confusion_matrix(labels_cat, preds_cat)

cum_err = np.sum(confusion, axis=1) - np.diag(confusion)

print("false pos: {}  false neg: {}, mis-class: {}".format(
    cum_err[0], np.sum(confusion[1:, 0]), np.sum(cum_err[1:]) - np.sum(confusion[1:, 0])))

In [ ]:
# filter short sequences

ji = np.mean([jaccard(onehot(l, np.arange(1, 20)), onehot(p, np.arange(1, 20)))
                 for l, p in zip (labels, preds)])
preds_cat = np.concatenate(preds)
labels_cat = np.concatenate(labels)

print("Jaccard index: {:0.3f}".format(ji))
print("Framewise: {:0.3f}".format(np.mean(preds_cat == labels_cat)))

thresholds = np.arange(0, 30)
jis = np.empty((len(thresholds),))
for i, t in enumerate(thresholds):
    preds2 = [gloss2seq([(g, start, stop)
                         for (g, start, stop) in seq2gloss(p) 
                         if stop - start > t],
                        len(p), 0)
              for p in preds]
    jis[i] = np.mean([jaccard(onehot(l, np.arange(1, 20)), onehot(p, np.arange(1, 20)))
                      for l, p in zip (labels, preds2)])

thres1 = thresholds[np.argmax(jis)]

thresholds = np.arange(30, 300, 5)
jis = np.empty((len(thresholds),))
for i, t in enumerate(thresholds):
    preds2 = [gloss2seq([(g, start, stop)
                         for (g, start, stop) in seq2gloss(p) 
                         if stop - start < t],
                        len(p), 0)
              for p in preds]
    jis[i] = np.mean([jaccard(onehot(l, np.arange(1, 20)), onehot(p, np.arange(1, 20)))
                      for l, p in zip (labels, preds2)])
thres2 = thresholds[np.argmax(jis)]

preds2 = [gloss2seq([(g, start, stop)
                     for (g, start, stop) in seq2gloss(p) 
                     if (stop - start > thres1) and (stop - start < thres2)],
                    len(p), 0)
          for p in preds]
ji = np.mean([jaccard(onehot(l, np.arange(1, 20)), onehot(p, np.arange(1, 20)))
              for l, p in zip (labels, preds2)])
preds_cat = np.concatenate(preds2)
labels_cat = np.concatenate(labels)
print("Optimal range: {}-{}".format(thres1, thres2))
print("Jaccard index: {:0.3f}".format(ji))
print("Framewise: {:0.3f}".format(np.mean(preds_cat == labels_cat)))

In [ ]:
# Plot the posteriors of the _correct_ states in color and other states in gray

def preview_seq(proba, gloss):
    plt.figure(figsize=(15, 3))
    cmap = plt.cm.summer(np.linspace(0, 1, 5))
    
    pre_start = 0
    for lbl, start, stop in gloss:
        for i in range(proba.shape[1] - 1):
            plt.plot(np.arange(pre_start, start), proba[pre_start:start, i], ls=':', c="gray")
        plt.plot(np.arange(pre_start, start), proba[pre_start:start, -1], c="purple")
        for a in range(0, (lbl - 1) * 5):
            plt.plot(np.arange(start, stop), proba[start:stop, a], ls=":", c='gray')
        for a in range(5):
            plt.plot(np.arange(start, stop), proba[start:stop, (lbl - 1) * 5 + a], c=cmap[a])
        for a in range(lbl * 5, proba.shape[1]):
            plt.plot(np.arange(start, stop), proba[start:stop, a], ls=":", c='gray')
        pre_start = stop
    
    for i in range(proba.shape[1] - 1):
        plt.plot(np.arange(pre_start, len(proba)), proba[pre_start:len(proba), i], ls=':', c="gray")
    plt.plot(np.arange(pre_start, len(proba)), proba[pre_start:len(proba), -1], c="purple")

    
seq = 15

preview_seq(
    recognizer.posterior.predict_proba(feats_seqs_val[seq][0]),
    gloss_seqs_val[seq])

plt.gca().set_xlim((200, 900))
plt.show()

## Test score

In [ ]:
# thres = <-set above

labels = [gloss2seq(g_, d_, 0)
          for g_, d_ in zip(gloss_seqs_test, durations_test)]

# Complete model
preds = recognizer.predict(feats_seqs_test)
preds2 = [gloss2seq([(g, start, stop)
                     for (g, start, stop) in seq2gloss(p) 
                     if (stop - start > thres1) and (stop - start < thres2)],
                    len(p), 0)
          for p in preds]

score = np.mean([jaccard(onehot(l, np.arange(1, 20)), onehot(p, np.arange(1, 20)))
                 for l, p in zip(labels, preds2)])

print("testing score: {}".format(score))

# Model

In [ ]:
print(recognizer.posterior.build_encoder.source)

In [ ]:
from sklearn.manifold import TSNE
from sltools.tconv import TemporalConv

l = None
for l_ in lasagne.layers.get_all_layers(recognizer.posterior.l_feats):
    if isinstance(l_, TemporalConv):
        l = l_

W1 = l.W.eval()
W1 = W1.transpose((0, 2, 1)).reshape((-1, W1.shape[1]))
Y = np.linalg.norm(W1, axis=1)
i = np.argsort(Y)
W1 = np.stack([W1[i_] for i_ in i])[-300:]
model = TSNE(n_components=1, metric='euclidean')
Y = model.fit_transform(W1)[:, 0]
i = np.argsort(Y)
W1 = W1[i]
Y = Y[i]

plt.figure(figsize=(5, 20))
x, y = np.meshgrid(np.arange(W1.shape[1]), Y)
plt.imshow(W1, clim=(-np.abs(W1).max(), np.abs(W1).max()), cmap='bwr')
plt.gca().set_aspect("auto")
plt.axis([-1, W1.shape[1], -1, W1.shape[0]])

In [ ]:
# l = lasagne.layers.get_all_layers(recognizer.posterior.l_feats)[2]
# W = np.asarray(l.W.eval())

# nrows, ncols = int(np.ceil(np.sqrt(W.shape[0] + 1))), int(np.floor(np.sqrt(W.shape[0] + 1)))
# img = np.zeros((nrows * (W.shape[2] + 1), ncols * (W.shape[3] + 1)))
# for k in range(W.shape[0]):
#     i, j = k // ncols, k % ncols
#     y, x = i * (W.shape[2] + 1), j * (W.shape[3] + 1)
#     tmp = img[y:y + W.shape[2]]
#     img[y:y + W.shape[2], x:x + W.shape[3]] = W[k, 0]

# plt.imshow(img, clim=(-np.abs(W).max(), np.abs(W).max()), cmap='bwr')
# plt.colorbar()

In [ ]:
lbl, cnt = np.unique(
    np.concatenate([gloss2seq(g_, len(r_), 0) for g_, r_ in zip(gloss_seqs_val, feats_seqs_val)]),
    return_counts=True)

In [ ]:
plt.bar(x=lbl+.5, height=cnt, log=True)
plt.gca().set_ylim((1, 3e5))

In [ ]:
plt.bar(x=lbl+.5, height=cnt, log=False)
plt.gca().set_ylim((1, 2e5))
plt.show()

In [ ]:
sum([len(gseq) for gseq in gloss_seqs_val])